Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## Define Strategy

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

In [2]:
class SimpleStrategy(Strategy):
    def __init__(self, asset, quantity):
        super().__init__()
        self.asset = asset
        self.quantity = quantity

    def log_all(self, orders, data, ctx, utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(utc)
            self.log_ohlcv(data, self.asset.symbol, ctx.exchange.id)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        quantity = .05
        price = data.get('close', self.asset.symbol, ctx.exchange.id)
        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, self.asset, quantity, price)
            orders.append(order)
        else:
            order = order_manager.build_market_sell_order(
                ctx.exchange, self.asset, quantity)
            orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []

        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)
        print(data.get('utc'))
        self.log_all(orders, data, ctx, data.get('utc'))
        return {
            'orders': orders,
            'cancel_ids': cancel_ids
        }

## Backtest

In [ ]:
cash = 1.0
ohlcv_fpath = os.path.join(cfg.DATA_DIR, c.DEFAULT_30M_FEED_CSV_FILENAME)
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.THIRTY_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = OHLCVFileFeed(
    fpath=ohlcv_fpath,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(balance, feed)
asset = Asset(c.ETH, c.BTC)
strategy = SimpleStrategy(asset, quantity=.05)
experiment_name = 'default_backtest'

In [ ]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

## Simulate Orders

In [3]:
cash = 1.0
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

assets = [Asset(c.ETH, c.BTC), Asset(c.LTC, c.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow()-datetime.timedelta(days=1),
    end=None
)
strategy = SimpleStrategy(assets[0], quantity=.05)

Downloading: ETH/BTC
Downloaded rows: 500
Downloading: LTC/BTC
Downloaded rows: 500
Downloading: BTC/USDT
Downloaded rows: 500


In [4]:
experiment_name = 'default_backtest_exchange'
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

Downloading: ETH/BTC
Downloaded rows: 500
Downloading: LTC/BTC
Downloaded rows: 500
Downloading: BTC/USDT
Downloaded rows: 500


---------------------------------------
Epoch 1 - Timestep: 2018-01-20T13:26:00
---------------------------------------
OHLCV
    O: 0.0897 | C: 0.0897 | V: 90.0 | T: 2018-01-20T13:26:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00


2018-01-20 13:26:00
Order Failed {
    "id": "238f393ae9344373bb84f97866a9f88f",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-21T00:38:12.335505",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "attempts": 0,
    "trades": [],
    "error": {
        "code": "INSUFFICIENT_FUNDS",
        "msg": "Insufficient funds to place order for asset ETH/BTC of cost: 0.00452. BTC balance is: 1.0. ETH balance is 0.0"
    },
    "symbol": "ETH/BTC"
}
Downloading: ETH/BTC
Downloaded rows: 500
Downloading: LTC/BTC
Downloaded rows: 500
Downloading: BTC/USDT
Downloaded rows: 500


---------------------------------------
Epoch 2 - Timestep: 2018-01-20T13:27:00
---------------------------------------
OHLCV
    O: 0.0897 | C: 0.0898 | V: 106.9 | T: 2018-01-20T13:27:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0898 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00


2018-01-20 13:27:00
Order Failed {
    "id": "238f393ae9344373bb84f97866a9f88f",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-21T00:38:12.335505",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "attempts": 1,
    "trades": [],
    "error": {
        "code": "INSUFFICIENT_FUNDS",
        "msg": "Insufficient funds to place order for asset ETH/BTC of cost: 0.00452. BTC balance is: 1.0. ETH balance is 0.0"
    },
    "symbol": "ETH/BTC"
}
Equal? True
Downloading: ETH/BTC
Downloaded rows: 500
Downloading: LTC/BTC
Downloaded rows: 500
Downloading: BTC/USDT
Downloaded rows: 500


...

---------------------------------------
Epoch 66 - Timestep: 2018-01-20T14:31:00
---------------------------------------
OHLCV
    O: 0.0895 | C: 0.0896 | V: 154.2 | T: 2018-01-20T14:31:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9338 Total Val: 1.0010 PnL: 0.0010 Returns: 0.0010
BALANCE
    BTC - {'free': 0.9338441999999998, 'used': 0.0, 'total': 0.9338441999999998}
    ETH - {'free': 0.7500000000000001, 'used': 0.0, 'total': 0.7500000000000001}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.7500000000000001, 'cost_price': 0.08950305732902211, 'latest_price': 0.08955}
METRICS
    SMA: 5.00
    RSI: 10.00


2018-01-20 14:31:00
Equal? True
Downloading: ETH/BTC
Downloaded rows: 500
Downloading: LTC/BTC
Downloaded rows: 500
Downloading: BTC/USDT
Downloaded rows: 500


---------------------------------------
Epoch 67 - Timestep: 2018-01-20T14:32:00
---------------------------------------
OHLCV
    O: 0.0896 | C: 0.0895 | V: 194.4 | T: 2018-01-20T14:32:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0895 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9384 Total Val: 1.0011 PnL: 0.0010 Returns: 0.0010
BALANCE
    BTC - {'free': 0.9383628499999997, 'used': 0.0, 'total': 0.9383628499999997}
    ETH - {'free': 0.7000000000000001, 'used': 0.0, 'total': 0.7000000000000001}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.7000000000000001, 'cost_price': 0.08950305732902211, 'latest_price': 0.089555}
METRICS
    SMA: 5.00
    RSI: 10.00


2018-01-20 14:32:00
Equal? True
Downloading: ETH/BTC
Downloaded rows: 500
Downloading: LTC/BTC
Downloaded rows: 500
Downloading: BTC/USDT


KeyboardInterrupt: 

In [ ]:
experiment_name = 'default_simulate'
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

## Live Trade

In [ ]:
exchange = load_exchange(c.BINANCE)
cash = exchange.fetch_balance().get(c.BTC)[BalanceType.FREE]
print(c.BTC, "cash", cash)

In [ ]:
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)

In [ ]:
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
asset = Asset(c.ETH, c.BTC)
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=[asset],
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy(asset, quantity=.05)
experiment_name = 'default_live'

In [ ]:
runner.live(experiment_name, exchange, balance, portfolio, feed, strategy)